<a href="https://colab.research.google.com/github/julialromero/Collaborative-Problem-Solving/blob/main/learning_pipeline_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import os
from skimage import io
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras import layers, Input, Model, optimizers
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

PROJECT_DIR = "drive/MyDrive/CSCI 5922 - Final Project/"  #<--- Make a shortcut for the "CSCI 5922 - Final Project" shared folder on your Google Drive

Thu Apr 28 21:11:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Get User Input

In [21]:
# Choose what size images to use
## "small" = 98x98, "med" = 332x332, "large" = 719x719
image_size = "med"
shuffled = False
chance = False
num_iters = 10
num_folds = 10
threshold = 0.5
num_epochs = 70

if image_size == "large":
  imgsize = (int(719/2), int(719/2)) 
  if shuffled:
    IMG_DIR = "719x719 - Recurrence_Matrices/RAW_SHUFFLED/"
  else:
    IMG_DIR = "719x719 - Recurrence_Matrices/RAW/"

elif image_size == "med":
  imgsize = (332, 332)
  if shuffled:
    IMG_DIR = "332x332 - Recurrence_Matrices/RAW_SHUFFLED/"
  else:
    IMG_DIR = "332x332 - Recurrence_Matrices/RAW/"

elif image_size == "small":
  imgsize = (98, 98)
  if shuffled:
    IMG_DIR = "98x98 - Recurrence_Matrices/RAW_SHUFFLED/"
  else:
    IMG_DIR = "98x98 - Recurrence_Matrices/RAW/"

print("Processing images from: ", IMG_DIR)

Processing images from:  332x332 - Recurrence_Matrices/RAW/


# Load Team Outcomes

In [5]:
outcome_df = pd.read_csv(PROJECT_DIR + "team_block_outcomes.csv")

# get median task_score
task_scores = outcome_df["task_score"]
median_task_score = np.median(task_scores)
mean_task_score = np.mean(task_scores)
print("median: ", median_task_score)
print("mean: ", mean_task_score)

# score_counts = dict(Counter(task_scores))
# for key in score_counts:
#     print("%d: %d" % (key, score_counts[key]))

outcome_df

median:  2.0
mean:  3.5437956204379564


,GROUPID,block,CPS_and_ITN_mean,Valence,num_gold,num_silver,task_score
0,1010,ExpBlock1,-0.304697,3.666667,1.0,1.0,3.0
1,1010,ExpBlock2,-0.304697,3.666667,2.0,1.0,5.0
2,1020,ExpBlock1,-0.389995,4.333333,0.0,2.0,2.0
3,1020,ExpBlock2,-0.026413,4.666667,4.0,1.0,9.0
4,1020,Warmup,-0.742380,3.666667,0.0,0.0,0.0
...,...,...,...,...,...,...,...
269,10103,ExpBlock2,-0.026413,3.666667,1.0,1.0,3.0
270,10103,Warmup,-1.405215,3.333333,1.0,2.0,4.0
271,10104,ExpBlock1,-0.531224,3.000000,0.0,0.0,0.0
272,10104,ExpBlock2,-1.023464,4.000000,0.0,1.0,1.0


# Helper Functions

In [22]:
# read, preprocess, and scale images to uniform dimension

def extract_image(image_path):
    imag = io.imread(image_path, as_gray=True)
    x = image.img_to_array(imag)

    x = tf.image.resize(x, imgsize) # TODO: try interpolation = "nearest" instead of default "bilinear"

    return x

def extract_image_inceptionv3(image_path):
    imag = io.imread(image_path, as_gray=True)
    x = image.img_to_array(imag)

    x = tf.image.resize(x, imgsize) # TODO: try interpolation = "nearest" instead of default "bilinear"
    x = np.squeeze(np.stack((x,)*3))
    x = x.swapaxes(0,1)
    x = x.swapaxes(1,2)

    return x

def extract_score(filename, outcome_df, score_col="task_score"):
    #  match this file's id/block with task score and return
    groupid, splitname = filename.split("-")
    block = splitname.split("_")[0]
    row = outcome_df.loc[(outcome_df.GROUPID == int(groupid)) & (outcome_df.block == block)]
    if(row.shape[0] != 1):
        print(f"Error -- Number of task scores recorded is not 1 for block {block} and groupid {groupid}.")
        return np.nan
    score = row[score_col].values[0]
    return score


def extract_binary_score(filename, outcome_df, score_col="task_score", median_score=2):
    #  match this file's id/block with task score and return
    groupid, splitname = filename.split("-")
    block = splitname.split("_")[0]
    row = outcome_df.loc[(outcome_df.GROUPID == int(groupid)) & (outcome_df.block == block)]
    if(row.shape[0] != 1):
        print(f"Error -- Number of task scores recorded is not 1 for block {block} and groupid {groupid}.")
        return np.nan
    raw_score = row[score_col].values[0]
    if raw_score <= median_score:
        score = 0
    else:
        score = 1
    return score



def get_fold_data(GROUPID_list, incv3=False, verbose=False):
  # read data to np arrays
  recurrence_plot_list = []
  if incv3:
    recurrence_plot_list_incv3 = []
  
  labels = []
  binary_labels = []

  for filename in os.listdir(PROJECT_DIR + IMG_DIR):
      # Check if file belongs to one of the teams in the desired group
      for GROUPID in GROUPID_list:
        if str(GROUPID) == filename.split("-")[0]:
          # preprocess image
          t = extract_image(PROJECT_DIR + IMG_DIR + filename)
          if incv3:
            t_incv3 = extract_image_inceptionv3(PROJECT_DIR + IMG_DIR + filename)

          # get task score
          lab = extract_score(filename, outcome_df, score_col="task_score")
          bin_lab = extract_binary_score(filename, outcome_df, score_col="task_score", median_score=2)

          # append data
          recurrence_plot_list.append(t)
          if incv3:
            recurrence_plot_list_incv3.append(t_incv3)
          
          labels.append(lab)
          binary_labels.append(bin_lab)
      
  recurrence_plot_list = np.array(recurrence_plot_list)
  if incv3:
    recurrence_plot_list_incv3 = np.array(recurrence_plot_list_incv3)
    plot_list = recurrence_plot_list_incv3
  else:
    plot_list = recurrence_plot_list
  
  labels = np.array(labels)
  binary_labels = np.array(binary_labels)

  if verbose:
    print("\m FOLD DATA INFO: ")
    print(f"\trecurrence_plot_list: {recurrence_plot_list.shape}")
    if incv3:
      print(f"\trecurrence_plot_list_incv3: {recurrence_plot_list_incv3.shape}")
    
    binary_counts = dict(Counter(binary_labels))
    print("\tBinary label counts: ", binary_counts)

  return [plot_list, labels, binary_labels]
  


def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

# Binary Classification w/ Team-level Cross Validation

## Prep folds for team-level cross validation

In [18]:
# Define fold names
train_folds = []
test_folds = []
set_type = "test"
for j in range(1,num_folds+1): 
    col_name = "Fold" + str(j) + "_" + set_type
    test_folds.append(col_name)
    set_type = "train"  
    col_name = "Fold" + str(j) + "_" + set_type
    train_folds.append(col_name)
    set_type = "test"

folds_dict_list = []

# Split teams into 5 groups
teams = pd.unique(outcome_df.GROUPID)

# For every iteration
for i in range(1,num_iters+1):
    print("Iteration: ", i)
    teams = shuffle(teams, random_state=i)
    groups = np.array_split(teams, num_folds)
    
    # Define groups for each fold
    fold_groups = {}
    for j, (train_fold, test_fold) in enumerate(zip(train_folds, test_folds)):
        # make the current group the test group
        fold_groups[test_fold] = groups[j]
        # make all other groups the train group
        train_group = groups[:j] + groups[j+1:]
        train_group = [team for group in train_group for team in group]
        fold_groups[train_fold] = train_group
        
    ## Confirm that for each fold, there is no team overlap bewteen train and test set
    for j in range(1,num_folds+1):
        assert set(fold_groups['Fold'+str(j)+'_test']).isdisjoint(set(fold_groups['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
    print("* No team overlap *")

      
    # Add fold groups to dictionary
    folds_dict_list.append(fold_groups)
    

# Informational
print("\nNumber of iterations: ", len(folds_dict_list))

print("\nIterating through folds_dict_list to check for overlap...")
for i,dicti in enumerate(folds_dict_list):
    for j in range(1,num_folds+1):
        assert set(dicti['Fold'+str(j)+'_test']).isdisjoint(set(dicti['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
print("* No team overlap *")  



Iteration:  1
* No team overlap *
Iteration:  2
* No team overlap *
Iteration:  3
* No team overlap *
Iteration:  4
* No team overlap *
Iteration:  5
* No team overlap *
Iteration:  6
* No team overlap *
Iteration:  7
* No team overlap *
Iteration:  8
* No team overlap *
Iteration:  9
* No team overlap *
Iteration:  10
* No team overlap *

Number of iterations:  10

Iterating through folds_dict_list to check for overlap...
* No team overlap *


## Prepare model

In [8]:
def get_binary_garcia_model():
  # Garcia-Ceja paper feeds CNN images with dims 100 x 100 x 4 (width x height x channels)
  # Labels are one hot encoded
  inputs = Input(shape=(imgsize[0], imgsize[1], 1))
  x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(inputs)
  x = layers.Conv2D(filters=16, kernel_size=3, strides=(1, 1), activation="relu")(x)
  x = layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = layers.Dropout(0.25)(x)
  x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
  x = layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), activation="relu")(x)
  x = layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = layers.Dropout(0.25)(x)
  x = layers.Dense(512, activation="relu")(x) 
  x = layers.Dropout(0.50)(x)
  x = layers.Flatten()(x)
  out = layers.Dense(1, activation="sigmoid")(x)   # change output layer to 1 and activation to sigmoid


  bc = tf.keras.losses.BinaryCrossentropy()        # change loss to binary
  lr = 0.00001 # paper used 0.001
  eps = 1e-08  # paper used 1e-08, but keras default is 1e-07
  adam = keras.optimizers.Adam(learning_rate=lr, epsilon=eps)
  auc = tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        from_logits=False
  )
  prec = tf.keras.metrics.Precision()
  rec = tf.keras.metrics.Recall()


  model_binary = Model(inputs=inputs, outputs=out)
  model_binary.compile(
    optimizer=adam,
    loss=bc, 
    metrics=['accuracy', auc, prec, rec],
  )

  return model_binary

model_binary = get_binary_garcia_model()
model_binary.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 98, 98, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 94, 94, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 47, 47, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 47, 47, 16)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 32)        4640      
                                                             

## Run team-level cross validation

In [ ]:
# Store metrics for all iterations
aurocs = []
precision = []
recall = []
accuracy = []

all_y_test_task_score = [[] for i in range(num_iters)]
predictions_task_score = [[] for i in range(num_iters)]
predict_proba_task_score = [[] for i in range(num_iters)]

# For each iteration 
for i in range(num_iters):
    print("Iteration: ", i+1)
    
    # Lists for cumulative predictions for iteration
    all_y_test = []
    predictions = []
    predict_proba = []
    
    # Get fold groups
    fold_groups = folds_dict_list[i]
    
    # For each fold
    for j, (test_fold, train_fold) in enumerate(zip(test_folds, train_folds)):
        print("\tFold: ", j+1)
        # Get data for teams in test set
        [plot_list, labels, binary_labels] = get_fold_data(fold_groups[test_fold], incv3=False)
        X_test = plot_list
        y_test = binary_labels
        all_y_test.extend(y_test.tolist())
        
        # Get data for teams in train set
        [plot_list, labels, binary_labels] = get_fold_data(fold_groups[train_fold], incv3=False)
        X_train = plot_list
        y_train = binary_labels

        # Train model
        model_binary = get_binary_garcia_model()
        # model_binary.summary()
        info_bin_cv = model_binary.fit(x=X_train, y=y_train, epochs=num_epochs, verbose=False)

        # Test model: https://androidkt.com/get-class-labels-from-predict-method-in-keras/
        y_pp = model_binary.predict(X_test)
        y_pred = np.where(y_pp > threshold, 1, 0)

        predict_proba.extend(y_pp.tolist())
        predictions.extend(y_pred.tolist())

    # ----- END OF FOLDS

    all_y_test = np.array(all_y_test)
    all_y_test_task_score[i] = all_y_test

    predictions = np.squeeze(np.array(predictions))
    predict_proba = np.squeeze(np.array(predict_proba))
    
    predictions_task_score[i] = predictions
    predict_proba_task_score[i] = predict_proba
    
    
    # Get metrics of iteration
    auroc = roc_auc_score(all_y_test, predict_proba)
    prec = precision_score(all_y_test, predictions)
    rec = recall_score(all_y_test, predictions)
    acc = accuracy_score(all_y_test, predictions)
    
    aurocs.append(auroc)
    precision.append(prec)
    recall.append(rec)
    accuracy.append(acc)

     # Save actual labels and predictions for iteration
    dfTruevPred = pd.DataFrame({'actual': all_y_test, 'predict_proba': predict_proba, 'prediction': predictions})

    if shuffled:
      ACTvPRED_SAVE_DIR = PROJECT_DIR + "results/shuffled/cross_val/" + image_size + "_TaskScore_True_vs_Pred_SHUFF_" + str(i+1) + ".csv"
    else:
      ACTvPRED_SAVE_DIR = PROJECT_DIR + "results/cross_val/" + image_size + "_TaskScore_True_vs_Pred_" + str(i+1) + ".csv"
    dfTruevPred.to_csv(ACTvPRED_SAVE_DIR, index=False)    
    
# ----- END OF ITERATIONS

print("\n =========== ALL ITERATIONS RESULTS SUMMARY ===========")
dfMetrics = pd.DataFrame({'iteration': [i for i in range(1,num_iters+1)], 'accuracy': accuracy, \
                          'auroc': aurocs, 'precision': precision, 'recall': recall})

display(dfMetrics)

if shuffled:
  METRICS_SAVE_DIR = PROJECT_DIR + "results/shuffled/cross_val/" + image_size + "_TaskScore_Metrics_SHUFF.csv"
else:
  METRICS_SAVE_DIR = PROJECT_DIR + "results/cross_val/" + image_size + "_TaskScore_Metrics.csv"
dfMetrics.to_csv(METRICS_SAVE_DIR, index=False)   

print("Averages: ")
print("%12s %.2f" % ("AUROC:", np.mean(dfMetrics['auroc'])))
print("%12s %.2f" % ("Precision:", np.mean(dfMetrics['precision'])))
print("%12s %.2f" % ("Recall:", np.mean(dfMetrics['recall'])))
print("%12s %.2f" % ("Accuracy:", np.mean(dfMetrics['accuracy'])))


print("\n%6s %.2f" % ("Med AUROC:", np.median(dfMetrics['auroc'])))

# Get median iterations
med_auroc = np.median(aurocs)
med_auroc_idx = np.argsort(aurocs)[len(aurocs)//2]

# Plot confusion matrix of median iteration
cf_matrix = confusion_matrix(all_y_test_task_score[med_auroc_idx], predictions_task_score[med_auroc_idx])
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Zero', 'One']
make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')
# TODO: Save confusion matrix


Iteration:  1
	Fold:  1
